## PAIR PROGRAMMING LIMPIEZA II

### Strings
---

In [66]:
import pandas as pd
import numpy as np
import re
pd.options.display.max_columns = None

**Preguntas:**

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?


¿Está limpia?
- age ❌      esta en formato string cuando debería ser integer y en algunos casos tenemos rangos de edad
- species ❌  es un jaleo! Debemos unificar los nombres y reducir a las especies más importantes
- country ✔️   los paises están en mayúsculas, algunos se repiten con algunos cambios
- fatal ✔️     la limpiamos en el pair de Pandas V
- year ✔️      es una columna de tipo float deberíamos convertirla a integer
- sex ✔️       la limpiamos en el pair de Pandas V
- fecha ✔️     la limpiamos en el pair de Pandas V
​

In [67]:
df= pd.read_pickle("../files/attacks8.pkl")
df.head(2)

,case_number,year,type,country,activity,age,species_,date,mes_ataque,fatal,sexo
0,1800.00.00,1800,Unprovoked,seychelles,a corsair's boat was overturned,NaN,NaN,1800,NaN,Y,F
1,1797.05.28.R,1797,Unprovoked,NaN,Dropped overboard,NaN,NaN,Reported May-28-1797,May,Y,NaN


**EJERCICIO 1**

Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. 

Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". 

Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

Imaginemos que el valor de una celda es el siguiente
​
'White shark, 3.5 m'
​
Tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 

Un patrón que podriamos usar es: 
​
patron_blanco = ```r".*[Ww](hite|HITE).*"``` # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 
​
De la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.


In [1]:
#list(df['species_'].unique()) Verificamos los valores unicos de la columna 'species_'

In [69]:
def especies(x):

    ''' Función para categorizar las especies de tiburones.
        Parámetros: valores de la columna seleccionada.
        Return: 5 tipos de tiburón o Unspecified y mantiene los valores nulos'''

    dict = {"White": ".*[Ww](hite|HITE).*", 
            "Tiger":".*[Tt](iger|IGER).*", 
            "Grey": ".*[Gg](rey|REY).*", 
            "Lemon": ".*[Ll](emon|EMON).*",
            "Bull": ".*[Bb](ull|ULL).*",
            "Unspecified": ".*"}

    try:
        for k,v in dict.items(): 
            if re.findall(v,x):           
                return k
               
    except:
        return np.nan

In [70]:
# Aplicamos la función a la columna species_ y lo comprobamos 
df["cat_species"] = df["species_"].apply(especies) 
df.head(2)

,case_number,year,type,country,activity,age,species_,date,mes_ataque,fatal,sexo,cat_species
0,1800.00.00,1800,Unprovoked,seychelles,a corsair's boat was overturned,NaN,NaN,1800,NaN,Y,F,NaN
1,1797.05.28.R,1797,Unprovoked,NaN,Dropped overboard,NaN,NaN,Reported May-28-1797,May,Y,NaN,NaN


In [71]:
# Comprobamos las categorias
df['cat_species'].value_counts() 

Unspecified    858
White          386
Tiger          157
Bull           104
Grey            30
Lemon           11
Name: cat_species, dtype: int64

In [72]:
# Comprobamos que mantenemos los valores nulos
df['cat_species'].isnull().sum()

126

In [73]:
# Borramos la columna y nos quedamos con la misma información mejor categorizada
df.drop(["species_"], axis=1, inplace=True)

In [74]:
df.head()

,case_number,year,type,country,activity,age,date,mes_ataque,fatal,sexo,cat_species
0,1800.00.00,1800,Unprovoked,seychelles,a corsair's boat was overturned,NaN,1800,NaN,Y,F,NaN
1,1797.05.28.R,1797,Unprovoked,NaN,Dropped overboard,NaN,Reported May-28-1797,May,Y,NaN,NaN
2,1792.09.12,1792,Provoked,england,Fishing,NaN,12-Sep-1792,Sep,Y,M,NaN
3,1791.00.00,1791,Unprovoked,australia,NaN,NaN,1791,NaN,Y,F,NaN
4,1788.05.10,1788,Boat,australia,Fishing,NaN,10-May-1788,May,N,NaN,NaN


---

**EJERCICIO 2**

Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:

- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?


Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.

Puede que os salga un error similar a este:
```python 
TypeError: expected string or bytes-like object
```
Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

df['nombre_columna'] = df['nombre_columna'].astype(str)

Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método .explode, 

Por último cambiad el tipo de la columna de string a integer.


In [75]:
df.head(2)

,case_number,year,type,country,activity,age,date,mes_ataque,fatal,sexo,cat_species
0,1800.00.00,1800,Unprovoked,seychelles,a corsair's boat was overturned,NaN,1800,NaN,Y,F,NaN
1,1797.05.28.R,1797,Unprovoked,NaN,Dropped overboard,NaN,Reported May-28-1797,May,Y,NaN,NaN


In [76]:
df['age'].unique()

array([nan, 'young', '14', '19', '20', '15', '21', '27', '36', '25', '6',
       '16', '50', '13 or 14', '57', '18', '32', '30', '60', '33', '10',
       '69', '55', '35', '54', '22', '31', '40', '37', '11', '13', '34',
       '46', '48', '17', '28', '65', '73', '58', '60s', '51', '61', '59',
       '42', '23', '29', '39', '24', '12', '26', '71', '43', '9', '44',
       '62', '52', '38', '68', '47', '63', '70', '41', '40s', '53', '20s',
       '7', '66', '45', '74', '64', '8', '56', '49', '18 or 20', 'Teen',
       '30s', '77', '8 or 10', '84', 'Â\xa0 ', ' ', '30 or 36', '6Â½',
       '5', ' 30', ' 28', "60's", '67', '>50', '? & 19', '21, 34,24 & 35',
       '30 & 32', '13 or 18', '7 or 8', '9 or 10'], dtype=object)

In [77]:
# Creamos un DataFrame con registros que hay en los casos en los que no tenemos una edad exacta o clara para comprobar la cantidad. 
# Razonando sobre que hacer en esos casos, pensamos que puede afectar si utilizamos el método 
# explode ya que añadiría más registros o filas influyendo en el análisis general.

df2 = df[(df['age'] == 'young') | (df['age'] ==  'Teen') | (df['age'] == 'Â\xa0 ')| (df['age'] =='21, 34,24 & 35')]
df2.head()

,case_number,year,type,country,activity,age,date,mes_ataque,fatal,sexo,cat_species
11,1779.00.00,1779,Unprovoked,usa,Surfing,young,1779,NaN,Y,M,NaN
630,2007.09.16.a,2007,Unprovoked,usa,Surfing,Teen,16-Sep-2007,Sep,N,F,Unspecified
854,2002.09.27.b,2002,Provoked,usa,Fishing,Â,27-Sep-2002,Sep,N,M,Unspecified
1510,1960.04.14,1960,Invalid,bermuda,Floating on a raft,"21, 34,24 & 35",14-Apr-1960,Apr,NaN,M,Unspecified


In [78]:
df.age.isnull().sum() # Comprobamos la cantidad de valores nulos de la columna 'age'

154

In [79]:
df.dtypes # Comprobamos que la columna 'age' es de tipo object-string

case_number    object
year            int64
type           object
country        object
activity       object
age            object
date           object
mes_ataque     object
fatal          object
sexo           object
cat_species    object
dtype: object

In [80]:
def edad(x):

       '''Función para sacar solo el número de edad de los registros con patrón de regex
          Parámetros: valores de la columna seleccionada.
          Return: el primer número de edad y los valores NaN'''

       patron_edad= '(\d*)'
       try: 
              return re.findall(patron_edad, x)[0]
       except:
              return np.nan


In [81]:
# Aplicamos la función a la columna 'age' creando una nueva, 'edad', con las nuevas categorías y lo comprobamos
df['edades'] = df['age'].apply(edad)
df.sample(2)

,case_number,year,type,country,activity,age,date,mes_ataque,fatal,sexo,cat_species,edades
942,2000.08.12,2000,Unprovoked,usa,Standing,44,12-Aug-2000,Aug,N,F,Unspecified,44
937,2000.09.08.a,2000,Unprovoked,reunion,Surfing,27,08-Sep-2000,Sep,N,M,Tiger,27


In [82]:
df['edades'].unique() # Comprobamos los valores resultantes

array([nan, '', '14', '19', '20', '15', '21', '27', '36', '25', '6', '16',
       '50', '13', '57', '18', '32', '30', '60', '33', '10', '69', '55',
       '35', '54', '22', '31', '40', '37', '11', '34', '46', '48', '17',
       '28', '65', '73', '58', '51', '61', '59', '42', '23', '29', '39',
       '24', '12', '26', '71', '43', '9', '44', '62', '52', '38', '68',
       '47', '63', '70', '41', '53', '7', '66', '45', '74', '64', '8',
       '56', '49', '77', '84', '5', '67'], dtype=object)

In [83]:
df.edades.isnull().sum() # Mantiene los mismos valores nulos

154

In [84]:
# Cambiamos el tipo de dato(object) de la columna 'edades' con el método pd.to_numeric()

df['edades'] = pd.to_numeric(df['edades'])

In [85]:
df.edades.dtypes #Comprobamos que el tipo de dato al que nos ha convertido la columna edades es de tipo float64, suponemos que es por los valores nulos

dtype('float64')

In [86]:
# Borramos la columna y nos quedamos con la misma información mejor categorizada
df.drop(["age"], axis=1, inplace=True)

In [87]:
df.head()

,case_number,year,type,country,activity,date,mes_ataque,fatal,sexo,cat_species,edades
0,1800.00.00,1800,Unprovoked,seychelles,a corsair's boat was overturned,1800,NaN,Y,F,NaN,NaN
1,1797.05.28.R,1797,Unprovoked,NaN,Dropped overboard,Reported May-28-1797,May,Y,NaN,NaN,NaN
2,1792.09.12,1792,Provoked,england,Fishing,12-Sep-1792,Sep,Y,M,NaN,NaN
3,1791.00.00,1791,Unprovoked,australia,NaN,1791,NaN,Y,F,NaN,NaN
4,1788.05.10,1788,Boat,australia,Fishing,10-May-1788,May,N,NaN,NaN,NaN


---

**EJERCICIO 3**

Guarda el archivo con las columnas limpias para seguir trabajando con este dataframe limpio.


In [88]:
df.to_pickle('../files/attacks9.pkl')